# Setup

In [1]:
from piper import piper
from piper.verbs import *
from piper.factory import get_sample_data

piper version 0.0.8, last run: Tuesday, 02 March 2021 09:52:35


# Importing data into a dataframe

In [2]:
%piper df <- get_sample_data()
%piper df >> head()

367 rows, 7 columns


,dates,order_dates,countries,regions,ids,values_1,values_2
0,2020-01-01,2020-01-07,Italy,East,A,311,26
1,2020-01-02,2020-01-08,Portugal,South,D,150,375
2,2020-01-03,2020-01-09,Spain,East,A,396,88
3,2020-01-04,2020-01-10,Italy,East,B,319,233


## compressing a column of values

In [3]:
g1 = df.groupby(['countries', 'regions', 'ids']).agg({
    'values_1': np.sum,
    'values_2': np.sum,
})
g1 = g1.unstack().fillna(0).astype(int)
head(g1, 10)

32 rows, 10 columns


values_1                        values_2                   \
ids                      A     B     C    D     E        A     B    C     D   
countries regions                                                             
France    East         253   404   426  700   387      612   496  581   617   
          North       1256   291   349  305    74     1292   825  278   384   
          South        776   308   304  347   383      732   522  102   277   
          West         796   566  1771  479  1249     1319   301  943   654   
Germany   East           0   458   282  578   446        0   408  363   510   
          North         64  1059   357    0   759      195   717  188     0   
          South        221     0   495  719   318       29     0  370  1267   
          West         524     0   178  353   520      433     0   13   173   
Italy     East         784   969   349  261   660      711  1153  269   187   
          North         87   847   609    0   325      203   795  442     0   

                         
ids                   E  
countries regions        
France    East      723  
          North     147  
          South      93  
          West      848  
Germany   East       81  
          North     636  
          South     362  
          West      702  
Italy     East     1443  
          North     486

In [4]:
%piper df <- get_sample_data()

grouper = df.groupby(['countries', 'regions'])
grouper = grouper.agg(listed_items=pd.NamedAgg('ids', list))
grouper.listed_items = grouper.listed_items.apply(sorted)
grouper = grouper.reset_index()
grouper.head(4)

,countries,regions,listed_items
0,France,East,"[A, A, A, B, B, C, C, C, D, D, D, E, E, E]"
1,France,North,"[A, A, A, A, A, B, B, B, C, C, D, E]"
2,France,South,"[A, A, A, A, B, B, C, D, E, E]"
3,France,West,"[A, A, A, A, A, B, B, C, C, C, C, C, C, D, D, ..."


In [5]:
df.query("countries == 'France' & regions == 'East'").sort_values('ids').head()

,dates,order_dates,countries,regions,ids,values_1,values_2
55,2020-02-25,2020-03-02,France,East,A,54,80
100,2020-04-10,2020-04-16,France,East,A,40,307
343,2020-12-09,2020-12-15,France,East,A,159,225
207,2020-07-26,2020-08-01,France,East,B,161,327
235,2020-08-23,2020-08-29,France,East,B,243,169


## first/top n group of records

In [6]:
grouping_columns = ['countries', 'regions', 'ids']
ranking_columns = ['countries', 'regions']

In [9]:
%%piper

get_sample_data()
>> group_by(grouping_columns)
>> summarise(totval1=('values_1', 'sum'))
>> group_calc(index=ranking_columns, value='totval1', function='percent', ascending=[True, False])
>> head()

145 rows, 2 columns


totval1  percent
countries regions ids                  
France    East    A        253    11.66
                  B        404    18.62
                  C        426    19.63
                  D        700    32.26

In [10]:
%piper df >> head()

367 rows, 7 columns


,dates,order_dates,countries,regions,ids,values_1,values_2
0,2020-01-01,2020-01-07,Italy,East,A,311,26
1,2020-01-02,2020-01-08,Portugal,South,D,150,375
2,2020-01-03,2020-01-09,Spain,East,A,396,88
3,2020-01-04,2020-01-10,Italy,East,B,319,233


In [11]:
df.to_excel('outputs/test.xlsx', index=False)